Shor's QECC upgraded (in parts)

In [2]:
from qiskit import QuantumRegister
from qiskit import ClassicalRegister
from qiskit import QuantumCircuit, execute,IBMQ
from qiskit.tools.monitor import job_monitor

#IBMQ.enable_account('0d1bdbd00494db146faab28ec991a0f5f9309d7cd2883abdacffef20da66e4c35770b8004aea4fad3ba3187b97a234d986d53e659e6fe4a5f27cd44ed8d616aa')
provider = IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('ibmq_qasm_simulator')

#We show that the initial state is the zero state
q = QuantumRegister(1,'q')
c = ClassicalRegister(1,'c')

#create the single qubit circuit and measure it's state
errorcircuit = QuantumCircuit(q,c)
errorcircuit.measure(q[0],c[0])

job = execute(errorcircuit, backend, shots=1000)
job_monitor(job)
counts = job.result().get_counts()

print("Measuring the qubit without operating on it", counts)
print("We measure the qubit in the 0 state")
print("The 1000 indicates that we ran this circuit 1000 times, to ensure accuracy")

#We show how the hadamard puts the qubit in a superposition of states with 50% chance to be measured in each state (the + state)
q = QuantumRegister(1,'q')
c = ClassicalRegister(1,'c')

errorcircuit = QuantumCircuit(q,c)
errorcircuit.h(q[0])                  #the hadamard gate
errorcircuit.measure(q[0],c[0])

job = execute(errorcircuit, backend, shots=1000)
job_monitor(job)
counts = job.result().get_counts()

print("The results of measuring the + state", counts)
print("We see each is measured roughly 50% of the time")

#We show how the hadamard of the plus state returns the qubit to its original state
q = QuantumRegister(1,'q')
c = ClassicalRegister(1,'c')

errorcircuit = QuantumCircuit(q,c)
errorcircuit.h(q[0])
errorcircuit.h(q[0])
errorcircuit.measure(q[0],c[0])

job = execute(errorcircuit, backend, shots=1000)
job_monitor(job)
counts = job.result().get_counts()

print("The original 0 state is seen again", counts)

Job Status: job has successfully run
Measuring the qubit without operating on it {'0': 1000}
We measure the qubit in the 0 state
The 1000 indicates that we ran this circuit 1000 times, to ensure accuracy
Job Status: job has successfully run
The results of measuring the + state {'0': 487, '1': 513}
We see each is measured roughly 50% of the time
Job Status: job has successfully run
The original 0 state is seen again {'0': 1000}


In [2]:
#We create a quantum circuit that has a bit flip error (without QECC)
q = QuantumRegister(1,'q')
c = ClassicalRegister(1,'c')

errorcircuit = QuantumCircuit(q,c)
errorcircuit.h(q[0])

#We create an error here 
errorcircuit.z(q[0])   #Phase flip error inside two hadamards is equivalent to a phase flip error

errorcircuit.h(q[0])
errorcircuit.barrier(q)
errorcircuit.measure(q[0],c[0])

job = execute(errorcircuit, backend, shots=1000)
job_monitor(job)
counts = job.result().get_counts()

print("Uncorrected bit flip error:", counts)
print("We see the error causes us to measure the output as a Qubit in the 1 state")

Job Status: job has successfully run
Uncorrected bit flip error: {'1': 1000}
We see the error causes us to measure the output as a Qubit in the 1 state


In [3]:
#We create a quantum circuit that has a phase flip error error (without QECC)
q = QuantumRegister(1,'q')
c = ClassicalRegister(1,'c')

errorcircuit = QuantumCircuit(q,c)
errorcircuit.h(q[0])

#We create an error here
errorcircuit.x(q[0])   #Bit flip error inside two hadamards is equivalent to a phase flip error

errorcircuit.h(q[0])
errorcircuit.barrier(q)
errorcircuit.measure(q[0],c[0])

job = execute(errorcircuit, backend, shots=1000)
job_monitor(job)
counts = job.result().get_counts()

print("Uncorrected phase error:", counts)
print("We notice there is no error, this is because phase flips do not effect the zero state")

Job Status: job has successfully run
Uncorrected phase error: {'0': 1000}
We notice there is no error, this is because phase flips do not effect the zero state


In [4]:
#We create a quantum circuit that has a bit flip and a phase flip error (without QECC)
q = QuantumRegister(1,'q')
c = ClassicalRegister(1,'c')

errorcircuit = QuantumCircuit(q,c)
errorcircuit.h(q[0])

#We create one of each error here for the code to correct
errorcircuit.x(q[0])   #Bit flip error
errorcircuit.z(q[0])   #Phase flip error

errorcircuit.h(q[0])
errorcircuit.barrier(q)
errorcircuit.measure(q[0],c[0])

job = execute(errorcircuit, backend, shots=1000)
job_monitor(job)
counts = job.result().get_counts()

print("Uncorrected bit flip and phase error:", counts)
print("We see the error causes us to measure the output as a Qubit in the 1 state")

Job Status: job has successfully run
Uncorrected bit flip and phase error: {'1': 1000}
We see the error causes us to measure the output as a Qubit in the 1 state


In [5]:
from qiskit import QuantumRegister
from qiskit import ClassicalRegister
from qiskit import QuantumCircuit, execute,IBMQ
from qiskit.tools.monitor import job_monitor

#IBMQ.enable_account('ibmq token')
provider = IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('ibmq_qasm_simulator')

#Shor Quantum Error Correcting Code
q = QuantumRegister(9,'q')
c = ClassicalRegister(1,'c')

#define the circuit
qecc = QuantumCircuit(q,c)

#first two CNOTs
qecc.cx(q[0],q[3])
qecc.cx(q[0],q[6])

#Hadamards put qubits in superposition
qecc.h(q[0])
qecc.h(q[3])
qecc.h(q[6])

#next set of CNOTs
qecc.cx(q[0],q[1])
qecc.cx(q[3],q[4])
qecc.cx(q[6],q[7])

qecc.cx(q[0],q[2])
qecc.cx(q[3],q[5])
qecc.cx(q[6],q[8])

qecc.barrier(q)

#The error is implimented here
qecc.x(q[0])   #The Bit flip error
qecc.z(q[0])   #The Phase flip error

#previous step with CNOTs repeated
qecc.barrier(q)
qecc.cx(q[0],q[1])
qecc.cx(q[3],q[4])
qecc.cx(q[6],q[7])

qecc.cx(q[0],q[2])
qecc.cx(q[3],q[5])
qecc.cx(q[6],q[8])

#Toffoli Gates applied
qecc.ccx(q[1],q[2],q[0])
qecc.ccx(q[4],q[5],q[3])
qecc.ccx(q[8],q[7],q[6])

#Hadamards to bring qubits out of superposition
qecc.h(q[0])
qecc.h(q[3])
qecc.h(q[6])

#final CNOT and Toffoli gates applied
qecc.cx(q[0],q[3])
qecc.cx(q[0],q[6])
qecc.ccx(q[6],q[3],q[0])

qecc.barrier(q)

#measuring the qubit to see if error was corrected
qecc.measure(q[0],c[0])

job = execute(qecc, backend, shots=1000)
job_monitor(job)
counts = job.result().get_counts()

print("\nShor code with bit flip and phase error:", counts)
print('We see the Error has been corrected and we now see the Qubit is in the 0 state')

Job Status: job has successfully run

Shor code with bit flip and phase error: {'0': 1000}
We see the Error has been corrected and we now see the Qubit is in the 0 state


In [15]:
from qiskit import QuantumRegister
from qiskit import ClassicalRegister
from qiskit import QuantumCircuit, execute,IBMQ
from qiskit.tools.monitor import job_monitor

#IBMQ.enable_account('ibmq token')
provider = IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('ibmq_qasm_simulator')

#Shor Quantum Error Correcting Code
q = QuantumRegister(9,'q')
c = ClassicalRegister(1,'c')

#define the circuit
qecc = QuantumCircuit(q,c)

#first two CNOTs
qecc.cx(q[0],q[3])
qecc.cx(q[0],q[6])

#Hadamards put qubits in superposition
qecc.h(q[0])
qecc.h(q[3])
qecc.h(q[6])

#next set of CNOTs
qecc.cx(q[0],q[1])
qecc.cx(q[3],q[4])
qecc.cx(q[6],q[7])

qecc.cx(q[0],q[2])
qecc.cx(q[3],q[5])
qecc.cx(q[6],q[8])

qecc.barrier(q)

#The error is implimented here
qecc.x(q[0])   #The Bit flip error on the first qubit 
qecc.z(q[0])   #The Phase flip error on the first qubit

#A second error is added here
#qecc.x(q[1])   #The Bit flip error on the second qubit 
qecc.z(q[3])   #The Phase flip error on the second qubit
#qecc.x(q[2])
#qecc.z(q[6])

#previous step with CNOTs repeated
qecc.barrier(q)
qecc.cx(q[0],q[1])
qecc.cx(q[3],q[4])
qecc.cx(q[6],q[7])

qecc.cx(q[0],q[2])
qecc.cx(q[3],q[5])
qecc.cx(q[6],q[8])

#Toffoli Gates applied
qecc.ccx(q[1],q[2],q[0])
qecc.ccx(q[4],q[5],q[3])
qecc.ccx(q[8],q[7],q[6])

#Hadamards to bring qubits out of superposition
qecc.h(q[0])
qecc.h(q[3])
qecc.h(q[6])

#final CNOT and Toffoli gates applied
qecc.cx(q[0],q[3])
qecc.cx(q[0],q[6])
qecc.ccx(q[6],q[3],q[0])

qecc.barrier(q)

#measuring the qubit to see if error was corrected
qecc.measure(q[0],c[0])

job = execute(qecc, backend, shots=1000)
job_monitor(job)
counts = job.result().get_counts()

print("\nShor code with a second error:", counts)
print('We see the Error has not been corrected and we measure the Qubit in the 1 state')

Job Status: job has successfully run

Shor code with a second error: {'1': 1000}
We see the Error has not been corrected and we measure the Qubit in the 1 state
